In [9]:
import timeit
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
with open('./Text.txt') as input_file:
    split_line = []
    for line in input_file:
        c_line = line.rstrip()
        split_line += c_line.split('=')

    keys = []
    values = []
    
    for i in range(0,len(split_line)):
        if i%2 ==0:
            keys.append(split_line[i].strip())
        else:
            values.append(split_line[i].strip())
    res = dict(map(lambda i,j : (i,j) , keys,values))

In [7]:
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id=res['SPOTIPY_CLIENT_ID'],
    client_secret = res['SPOTIPY_CLIENT_SECRECT']
    )
)

In [19]:
# timeit library to measure the time needed to run this code

start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0, 1000, 50):
    track_results = sp.search(q='year:2023', type='track', limit=50, offset=i)

    for t in track_results['tracks']['items']:
        release_year = int(t['album']['release_date'][:4])  # Extract the year from the release date and convert to integer
        if 2017 <= release_year <= 2023:
            artist_name.append(t['artists'][0]['name'])
            track_name.append(t['name'])
            track_id.append(t['id'])
            popularity.append(t['popularity'])


stop = timeit.default_timer()
print('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 11.247946090064943


In [24]:
len(track_results['tracks']['items'])

50

In [ ]:
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id=res['SPOTIPY_CLIENT_ID'],
    client_secret = res['SPOTIPY_CLIENT_SECRECT']
    )
)

results = spotify.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])

In [ ]:
def get_tracks_data(spotify_client, query, chunk_size, max_results):

    tracks_data_list = []
    single_track_dict = {}

    for i in range(0, max_results, chunk_size):
        try:
            track_results = spotify_client.search(q=query, type='track', limit=50,offset=i)
            for i, track in enumerate(track_results['tracks']['items']):                              
                single_track_dict = {                                       
                    'track_name': track['name'],
                    'artist_name': track['artists'][0]['name'],
                    'track_id': track['id']            
                }
                tracks_data_list.append(single_track_dict)
        except Exception as e:
            print(e)
            logger.info(e)
            continue

    return tracks_data_list

In [ ]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [ ]:
def get_audio_features_data(spotify_client, track_ids, chunk_size):
    audio_features_list = []

    for i in range(0, len(track_ids), chunk_size):    
        try:
            track_id_list = track_ids[i:i+chunk_size]
            results = spotify_client.audio_features(track_id_list)
            results = [dict({'id':'None'}) if v is None else v for v in results]
            audio_features_list.extend(results)
        except Exception as e:
            print(e)
            logger.info(e)
            continue
    return audio_features_list

In [ ]:
def insert_bigquery(client, project_id, dataset_id, table_id, data):

    table_id = "{}.{}.{}".format(project_id, dataset_id, table_id)
    job_config = bigquery.LoadJobConfig(autodetect=True, write_disposition=bigquery.WriteDisposition.WRITE_APPEND, schema_update_options = bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION)
    job = client.load_table_from_json(destination=table_id, json_rows=data, job_config=job_config)

    logger.info("Successfully uploaded to table {}".format(table_id))


In [ ]:
def get_spotify_audio_features_data_to_bigquery(event, context):
    """
    This function combines data from 2 Spotify endpoints:
    1. v1/search
    2. v1/audio-features
    and loads it into BigQuery
    """

    client_id = os.environ['CID']
    client_secret = os.environ['SECRET']
    project_id = os.environ['PROJECT_ID']
    dataset_id = os.environ['DATASET_ID']
    table_id = os.environ['TABLE_ID']

    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    spotify_client = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
        
    tracks_data_list = get_tracks_data(spotify_client, 'year:2021', 50, 1000)
    track_ids = [d['track_id'] for d in tracks_data_list]
    audio_features_list = get_audio_features_data(spotify_client, track_ids, 100)
    merged_list = [{**u, **v} for u, v in zip(tracks_data_list, audio_features_list)]
    
    client = bigquery.Client()

    insert_bigquery(client, project_id, dataset_id, table_id, merged_list)